<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Lab: Working with a real world data-set using SQL and Python</font></h1>

# Introduction

This notebook shows how to work with a real world dataset using SQL and Python. In this lab you will:
1. Understand the dataset for Chicago Public School level performance 
1. Store the dataset in an Db2 database on IBM Cloud instance
1. Retrieve metadata about tables and columns and query data from mixed case columns
1. Solve example problems to practice your SQL skills including using built-in database functions

## Chicago Public Schools - Progress Report Cards (2011-2012) 

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true

Now download a static copy of this database and review some of its contents:
https://ibm.box.com/shared/static/0g7kbanvn5l2gt2qu38ukooatnjqyuys.csv


### Store the dataset in a Table
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called __SCHOOLS__.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>

### Connect to the database
Let us now load the ipython-sql  extension and establish a connection with the database

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://wjr95037:8bj4w+93nmtvksfq@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB

'Connected: wjr95037@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created

In [12]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT * from syscat.tables where tabname='SCHOOLS'; 

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
WJR95037,SCHOOLS,WJR95037,U,T,N,None,None,None,None,2018-12-14 07:03:20.092096,2018-12-14 07:03:20.092096,2018-12-14 07:03:20.092096,2018-12-14 07:08:26.892580,79,12,889,566,15,0,16,-1,-1,-1,0,wjr95037space1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,850,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2018-12-14 07:03:20.092096,0,,None,None,N,WJR95037,,,,2018-12-14,,N,R,N,-1.0,None


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->

### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?

In [15]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql SELECT count(*) from syscat.columns where tabname='SCHOOLS';

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
79


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->

Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [16]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT * from syscat.columns where tabname='SCHOOLS';

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,colname,colno,typeschema,typename,length,scale,typestringunits,stringunitslength,DEFAULT,NULLS,codepage,collationschema,collationname,logged,compact,colcard,high2key,low2key,avgcollen,keyseq,partkeyseq,nquantiles,nmostfreq,numnulls,target_typeschema,target_typename,scope_tabschema,scope_tabname,source_tabschema,source_tabname,dl_features,special_props,hidden,inline_length,pctinlined,IDENTITY,rowchangetimestamp,GENERATED,text,compress,avgdistinctperpage,pagevarianceratio,sub_count,sub_delim_length,avgcollenchar,implicitvalue,seclabelname,rowbegin,rowend,transactionstartid,pctencoded,avgencodedcollen,qualifier,func_path,randdistkey,remarks
WJR95037,SCHOOLS,School_ID,0,SYSIBM,INTEGER,4,0,None,None,None,Y,0,None,None,,,566,610543,609674,5,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,Name_of_School,1,SYSIBM,VARCHAR,65,0,OCTETS,65,None,Y,1208,SYSIBM,IDENTITY,,,566,'Woodlawn Community Elementary Sch','Abraham Lincoln Elementary School',41,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,36,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,"Elementary, Middle, or High School",2,SYSIBM,VARCHAR,2,0,OCTETS,2,None,Y,1208,SYSIBM,IDENTITY,,,3,'MS','ES',7,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,2,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,Street_Address,3,SYSIBM,VARCHAR,30,0,OCTETS,30,None,Y,1208,SYSIBM,IDENTITY,,,566,'9912 S Avenue H','10015 S Leavitt St',22,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,17,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,City,4,SYSIBM,VARCHAR,7,0,OCTETS,7,None,Y,1208,SYSIBM,IDENTITY,,,1,'Chicago','Chicago',12,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,7,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,State,5,SYSIBM,VARCHAR,2,0,OCTETS,2,None,Y,1208,SYSIBM,IDENTITY,,,1,'IL','IL',7,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,2,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,ZIP_Code,6,SYSIBM,INTEGER,4,0,None,None,None,Y,0,None,None,,,51,60707,60607,5,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,Phone_Number,7,SYSIBM,VARCHAR,14,0,OCTETS,14,None,Y,1208,SYSIBM,IDENTITY,,,566,'(773) 535-9930','(773) 534-0146',19,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,14,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,Link,8,SYSIBM,VARCHAR,78,0,OCTETS,78,None,Y,1208,SYSIBM,IDENTITY,,,2,'','http://schoolreports.cps.edu/Scho',83,None,0,20,10,1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,78,None,None,N,N,N,-1,-1.0,None,None,N,None
WJR95037,SCHOOLS,Network_Manager,9,SYSIBM,VARCHAR,40,0,OCTETS,40,None,Y,1208,SYSIBM,IDENTITY,,,20,'Southwest Side High School Networ','Austin-North Lawndale Elementary ',35,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,30,None,None,N,N,N,-1,-1.0,None,None,N,None


Double-click __here__ for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->

### Questions
1. Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
1. What is the "Community Area Name" field called in your table? Have the spaces " " between the words been replaced by some other character?
1. Have the paranthesis (round brackets) in the "College Enrollment (number of students)" attribute been replaced by some other character? 
1. Are there any columns in whose names the spaces and paranthesis (round brackets) have not been replaced by the underscore character "_"?

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?

In [18]:
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
462


Double-click __here__ for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'?
-->

Double-click __here__ for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->

### Problem 2

##### What is the highest Safety Score?

In [21]:
%sql select max("Safety_Score") from SCHOOLS;

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


1
99


Double-click __here__ for a hint

<!--
Use the MAX() function
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql select MAX("Safety_Score") AS MAX_SAFETY_SCORE from SCHOOLS
Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?

In [22]:
%sql select "Name_of_School", "Safety_Score" from SCHOOLS where "Safety_Score" = 99

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Name_of_School,Safety_Score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click __here__ for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select "Name_of_School", "Safety_Score" from SCHOOLS where "Safety_Score" = 99

or, a better way:

%sql select "Name_of_School", "Safety_Score" from SCHOOLS where \
  "Safety_Score"= (select MAX("Safety_Score") from SCHOOLS)

Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [27]:
%sql select "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
    order by "Average_Student_Attendance" desc limit 10 

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Name_of_School,Average_Student_Attendance
Velma F Thomas Early Childhood Center,None
John Charles Haines Elementary School,98.4%
James Ward Elementary School,97.8%
Edgar Allan Poe Elementary Classical School,97.6%
Rachel Carson Elementary School,97.6%
Orozco Fine Arts & Sciences Elementary School,97.6%
Annie Keller Elementary Gifted Magnet School,97.5%
Andrew Jackson Elementary Language Academy,97.4%
Lenart Elementary Regional Gifted Center,97.4%
Disney II Magnet School,97.3%


Double-click __here__ for the solution.

<!-- Solution:

%sql select "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
    order by "Average_Student_Attendance" desc nulls last limit 10 

-->

### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [30]:
%sql SELECT "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
     order by "Average_Student_Attendance" asc limit 5;
#rows only

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Name_of_School,Average_Student_Attendance
Richard T Crane Technical Preparatory High School,57.9%
Barbara Vick Early Childhood & Family Center,60.9%
Dyett High School,62.5%
Wendell Phillips Academy High School,63.0%
Orr Academy High School,66.3%


Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column

In [31]:
%sql SELECT "Name_of_School", REPLACE("Average_Student_Attendance", '%', '') \
     from SCHOOLS order by "Average_Student_Attendance" asc limit 5
#rows only

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Name_of_School,2
Richard T Crane Technical Preparatory High School,57.9
Barbara Vick Early Childhood & Family Center,60.9
Dyett High School,62.5
Wendell Phillips Academy High School,63.0
Orr Academy High School,66.3


Double-click __here__ for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql SELECT "Name_of_School", REPLACE("Average_Student_Attendance", '%', '') \
     from SCHOOLS \
     order by "Average_Student_Attendance" \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?

In [41]:
%sql SELECT "Name_of_School", "Average_Student_Attendance" from SCHOOLS \
     where CAST ( REPLACE("Average_Student_Attendance", '%', '') AS DOUBLE ) < 70 \
     order by "Average_Student_Attendance"

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Name_of_School,Average_Student_Attendance
Richard T Crane Technical Preparatory High School,57.9%
Barbara Vick Early Childhood & Family Center,60.9%
Dyett High School,62.5%
Wendell Phillips Academy High School,63.0%
Orr Academy High School,66.3%
Manley Career Academy High School,66.8%
Chicago Vocational Career Academy High School,68.8%
Roberto Clemente Community Academy High School,69.6%


Double-click __here__ for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->

Double-click __here__ for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     where CAST ( REPLACE("Average_Student_Attendance", '%', '') AS DOUBLE ) < 70 \
     order by "Average_Student_Attendance"
     
or,

%sql SELECT "Name_of_School", "Average_Student_Attendance"  \
     from SCHOOLS \
     where DECIMAL ( REPLACE("Average_Student_Attendance", '%', '') ) < 70 \
     order by "Average_Student_Attendance"

-->


### Problem 8

##### Get the total __College Enrollment (number of students)__ for each Community Area

In [37]:
%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS group by "Community_Area_Name"

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Community_Area_Name,total_enrollment
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175
AUSTIN,10933
AVALON PARK,1522
AVONDALE,3640
BELMONT CRAGIN,14386
BEVERLY,1636


Double-click __here__ for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->

Double-click __here__ for another hint

<!--
Don't forget to group by the Community Area
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by "Community_Area_Name" 

-->


### Problem 9

##### Get the 5 Community Areas with the least total __College Enrollment (number of students)__  sorted in ascending order 

In [40]:
%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS group by "Community_Area_Name" \
   order by TOTAL_ENROLLMENT asc fetch first 5 rows only

 * ibm_db_sa://wjr95037:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Community_Area_Name,total_enrollment
OAKLAND,140
FULLER PARK,531
BURNSIDE,549
OHARE,786
LOOP,871


Double-click __here__ for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select "Community_Area_Name", sum("College_Enrollment__number_of_students_") AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by "Community_Area_Name" \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed names. You also used built in database functions.

Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).


In [ ]:
head